# Retrieving data from Kiva API

We will be using [KIVA](https://www.kiva.org/) data for the entirety of this course. Kiva makes it's data publically available through it's [API](http://build.kiva.org/). 

Not sure what an API is? It stands for Application Program Interface. **To a data scientist, it's an incredible way for you to access and selectively pull data.** Whereas many APIs are not open for public access, Kiva's is, giving us wide access to rich datasets.  

For more on APIs, CodeAcademy has a fantastic short course (link [here](https://www.codecademy.com/en/tracks/placekitten) that introduces you to APIs and allows you to pull images of kittens from a website by the end of the session.

Below, we import the packages we need in order to retrieve data from the API.

In [2]:
from urllib.request import urlopen, Request
import json
from pandas.io.json import json_normalize
import pandas as pd
import requests as r
import os
import logging
import time
pd.set_option('display.max_columns', 80)

The cell below enables Jupyter to show the output of multiple cells at once. There are a lot of cool jupyter specific commands to allow you to work faster. [Here](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/) is a great article that goes through some less well known shortcuts and tricks.

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In the cell below we set the maximum number of columns to 100 so we can see our entire data set. If we did not set this option, we would not be able to see some columns in our dataset.

In [4]:
pd.set_option('display.max_columns', 100) 

Using % in the section below indicates a magic command. You can find out more about magic commands [here](https://ipython.org/ipython-doc/3/interactive/magics.html). To see all available magic commands run this line of code:

`%lsmagic`

Using the % in the block of code below allows us to run commands on the terminal from within jupyter. We make a folder in our user directory where we will store the data we retrieve from the api.

In [8]:
import os 
#os.mkdir("~/intro_to_machine_learning")
os.chdir("C:\\Users\h\Desktop\kiva_data")
% pwd
% ls

'C:\\Users\\h\\Desktop\\kiva_data'

 Volume in drive C is Windows
 Volume Serial Number is 38A2-E8A5

 Directory of C:\Users\h\Desktop\kiva_data

05/30/2017  09:53 AM    <DIR>          .
05/30/2017  09:53 AM    <DIR>          ..
05/30/2017  11:49 PM             1,024 kiva_cache.h5
               1 File(s)          1,024 bytes
               2 Dir(s)  18,004,729,856 bytes free


In the line below, we create a data cache. A cache is a great way to store data, when it is costly to retrive it from scratch every time.

In [9]:
store = pd.HDFStore('kiva_cache.h5')

There is great documentation on Kiva's API [here](http://build.kiva.org/api). The documentation explains what parameters (conditions) we need to pass in our request to Kiva's database in order to get the data we want.

We are trying to retrive all Kiva data from Kenya. So we will be using two main parameters where we set country_code=KE (KE is the two letter [ISO](https://en.wikipedia.org/wiki/ISO_3166-2) code for Kenya), and we increase the results per page to 500 (this is the maximum KIVAs API appears to allow). You can see the HTML results of the api call by pasting the url below into your browser, HTML is a format that is really easy to read.

http://api.kivaws.org/v1/loans/search/?country_code=KE&per_page=500

1) Go ahead and play with the url in order to retrieve different data. For example, how would you retrieve data from South Africa (ZA)?

2) How would you only retrive 200 results?

Answers:

1) http://api.kivaws.org/v1/loans/search/?country_code=ZA&per_page=500

2)  http://api.kivaws.org/v1/loans/search/?country_code=ZA&per_page=200



We want to request this data from the api and store it in a format that is more intuitive to us - a dataframe. Let's get started. The code below retrieves the first 500 results and converts it into a pandas dataframe. You will get to know a lot more about dataframes over the next few classes.

In [52]:
d = r.get('http://api.kivaws.org/v1/loans/search.json?country_code=KE&per_page=500')

Notice that in the request above we specify json as the type of text we want returned. This is easier to handle and change into a python dataframe. You can past the link into your browser to understand the difference between [JSON](https://en.wikipedia.org/wiki/JSON) and [HTML](https://en.wikipedia.org/wiki/HTML).

By running d.headers below we can see all the data associated with our request. It shows the time of our request, the fact that we are requesting json text 'Content-Type': 'application/json; charset=UTF-8', in addition to other details.

In [53]:
d.headers

{'Date': 'Wed, 03 May 2017 23:56:20 GMT', 'Server': 'Apache/2.4.7 (Ubuntu)', 'Access-Control-Allow-Origin': '*', 'Expires': 'Tue, 03 Jul 2001 06:00:00 GMT', 'Last-Modified': 'Wed, 03 May 2017 23:56:20 GMT', 'Cache-Control': 'private, no-store, no-cache, must-revalidate, max-age=0, post-check=0, pre-check=0, proxy-revalidate, no-transform', 'Pragma': 'no-cache', 'X-RateLimit-Overall-Limit': '60', 'X-RateLimit-Overall-Remaining': '60', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '26111', 'Content-Type': 'application/json; charset=UTF-8'}

By running the command d.json() below we can get an idea of what our data looks like before we change it into a pandas data frame. In order to see the output, remove the `;` from the end of the codeblock. We have added the `;` to supress output for easier reading in our repo.

In [222]:
d.json()

{'code': 'org.kiva.RateLimitExceeded',
 'message': 'You are making requests to the server too often and have been temporarily blocked. You can find out more details at http://build.kiva.org/docs/data/rate.'}

`json.loads` turns the json object into a python object.

In [60]:
data = json.loads(d.text)

`json_normalize` normalizes semi-structured json data into a flat table

In [61]:
loans=json_normalize(data['loans'])

We have now extracted the first 500 rows of loans from the API. We can confirm how many rows we have in a dataset using the len() function below. Now, we need to extract this data more systematically for all Kenyan loan results. KIVA has a parameter called page but does not allow for range of pages, so we will have to create a python loop to go through each page of results and add to our dataset.

In [63]:
len(loans.index)

500

In the function defined below we are doing a few things. Firstly, we create an empty dataframe, then we loop through each page and call the api. We add the results of each page to our dataframe.

The first part of the code extracts all the loans in Kenya, the second part of the function extracts additional loan information (like the longer description about the loan) by making a second api call. This is inserted in a second dataframe called loans_details. There is a restriction on the second API call which only allows us to pass 100 loans in one call. To address this, we have a parameter in `pages=100` which means we only extract 100 loans in every call which allows us to satisfy the second restriction.

In [286]:
def extract_loans(pages,country_iso_code):
    loans_full=pd.DataFrame()
    loans_details=pd.DataFrame()
    for n in range(1, pages+1):
        s=str(n)
        print(s)
        d = r.get('http://api.kivaws.org/v1/loans/search.json?country_code='+country_iso_code+'&per_page=100&page='+s)
        data = json.loads(d.text)
        loans=json_normalize(data['loans'])
        
        #below we take the loan id column from our retrieved loan data, and use it to pull additional details
        #about our loans. We store this in a seperate table loans_details
        
        loan_id= loans['id'].tolist()
        loan_id_str = ','.join(str(e) for e in loan_id)
        ld=r.get('https://api.kivaws.org/v1/loans/'+loan_id_str+'.json')
        ld_data = json.loads(ld.text)
        loans_ext=json_normalize(ld_data['loans'])
        loans_details=loans_details.append(loans_ext, ignore_index=True)
        
        loans_full=loans_full.append(loans,ignore_index=True)
        print(len(loans_full.index))
        time.sleep(10)
        
    return loans_full, loans_details

In [287]:
loans_full,loans_details = extract_loans(1279,'KE')

1
100
2
200
3
300
4
400
5
500
6
600
7
700
8
800
9
900
10
1000
11
1100
12
1200
13
1300
14
1400
15
1500
16
1600
17
1700
18
1800
19
1900
20
2000
21
2100
22
2200
23
2300
24
2400
25
2500
26
2600
27
2700
28
2800
29
2900
30
3000
31
3100
32
3200
33
3300
34
3400
35
3500
36
3600
37
3700
38
3800
39
3900
40
4000
41
4100
42
4200
43
4300
44
4400
45
4500
46
4600
47
4700
48
4800
49
4900
50
5000
51
5100
52
5200
53
5300
54
5400
55
5500
56
5600
57
5700
58
5800
59
5900
60
6000
61
6100
62
6200
63
6300
64
6400
65
6500
66
6600
67
6700
68
6800
69
6900
70
7000
71
7100
72
7200
73
7300
74
7400
75
7500
76
7600
77
7700
78
7800
79
7900
80
8000
81
8100
82
8200
83
8300
84
8400
85
8500
86
8600
87
8700
88
8800
89
8900
90
9000
91
9100
92
9200
93
9300
94
9400
95
9500
96
9600
97
9700
98
9800
99
9900
100
10000
101
10100
102
10200
103
10300
104
10400
105
10500
106
10600
107
10700
108
10800
109
10900
110
11000
111
11100
112
11200
113
11300
114
11400
115
11500
116
11600
117
11700
118
11800
119
11900
120
12000
121
12100
122
12

In [289]:
loans_full.to_csv('~/intro_to_machine_learning/data/loans_full_large.csv')
loans_details.to_csv('~/intro_to_machine_learning/data/loans_details.csv')

We will also pull data about Kiva's partners, who are the intermediaries between Kiva and end recipients for the majority of loans made.

In [2]:
data_path = '~/intro_to_machine_learning/data'

In [52]:
df=pd.read_csv(data_path+'/loans_details.csv', low_memory=False)

In [53]:
df.head()

,Unnamed: 0,activity,basket_amount,bonus_credit_eligibility,borrowers,currency_exchange_loss_amount,description.languages,description.texts.en,description.texts.es,description.texts.fr,description.texts.ru,funded_amount,funded_date,id,image.id,image.template_id,journal_totals.bulkEntries,journal_totals.entries,lender_count,loan_amount,location.country,location.country_code,location.geo.level,location.geo.pairs,location.geo.type,location.town,name,partner_id,payments,planned_expiration_date,posted_date,sector,status,tags,terms.disbursal_amount,terms.disbursal_currency,terms.disbursal_date,terms.loan_amount,terms.local_payments,terms.loss_liability.currency_exchange,terms.loss_liability.currency_exchange_coverage_rate,terms.loss_liability.nonpayment,terms.repayment_interval,terms.repayment_term,terms.scheduled_payments,themes,translator.byline,translator.image,use,video.id,video.thumbnailImageId,video.title,video.youtubeId
0,0,Farming,0.0,False,"[{'first_name': 'Evaline', 'last_name': '', 'g...",NaN,['en'],Evaline is a married lady aged 44 years old an...,NaN,NaN,NaN,0,NaN,1291548,2516002,1,0,0,0,500,Kenya,KE,town,-0.583333 35.183333,point,litein,Evaline,386.0,[],2017-06-08T00:40:03Z,2017-05-09T00:40:03Z,Agriculture,fundraising,"[{'name': '#Woman Owned Biz'}, {'name': '#Pare...",50000.0,KES,2017-04-03T07:00:00Z,500,"[{'due_date': '2017-05-10T07:00:00Z', 'amount'...",shared,0.1,lender,Monthly,14,"[{'due_date': '2017-07-01T07:00:00Z', 'amount'...",NaN,Julie Keaton,892591.0,to purchase more tea leaves to sell to the tea...,NaN,NaN,NaN,NaN
1,1,Furniture Making,0.0,False,"[{'first_name': 'Julias', 'last_name': '', 'ge...",NaN,['en'],Aged 42 years is a man by the name of Julias. ...,NaN,NaN,NaN,0,NaN,1291532,2515992,1,0,0,0,500,Kenya,KE,town,0.566667 34.566667,point,Bungoma,Julias,386.0,[],2017-06-08T00:30:05Z,2017-05-09T00:30:05Z,Manufacturing,fundraising,[],50000.0,KES,2017-04-03T07:00:00Z,500,"[{'due_date': '2017-05-09T07:00:00Z', 'amount'...",shared,0.1,lender,Monthly,14,"[{'due_date': '2017-07-01T07:00:00Z', 'amount'...",NaN,Morena Calvo,1832928.0,to buy timber to make more furniture for his e...,NaN,NaN,NaN,NaN
2,2,Home Energy,0.0,False,"[{'first_name': 'Rose', 'last_name': '', 'gend...",NaN,['en'],"Hello Kiva Community! <br /><br />Meet Rose, w...",NaN,NaN,NaN,50,NaN,1291530,2515991,1,0,0,2,75,Kenya,KE,town,0.516667 35.283333,point,Eldoret,Rose,156.0,[],2017-06-08T00:30:03Z,2017-05-09T00:30:04Z,Personal Use,fundraising,"[{'name': '#Eco-friendly'}, {'name': '#Technol...",6000.0,KES,2017-04-28T07:00:00Z,75,"[{'due_date': '2017-05-14T07:00:00Z', 'amount'...",shared,0.1,lender,Monthly,14,"[{'due_date': '2017-07-01T07:00:00Z', 'amount'...","['Green', 'Earth Day Campaign']",Julie Keaton,892591.0,to buy a solar lantern.,NaN,NaN,NaN,NaN
3,3,Used Clothing,0.0,False,"[{'first_name': 'Jane', 'last_name': '', 'gend...",NaN,['en'],"Jane was born in the 1980, and she is happily ...",NaN,NaN,NaN,0,NaN,1291525,2515986,1,0,0,0,500,Kenya,KE,town,0.566667 34.566667,point,Bungoma,Jane,386.0,[],2017-06-08T00:20:04Z,2017-05-09T00:20:04Z,Clothing,fundraising,[{'name': '#Eco-friendly'}],50000.0,KES,2017-04-03T07:00:00Z,500,"[{'due_date': '2017-05-08T07:00:00Z', 'amount'...",shared,0.1,lender,Monthly,14,"[{'due_date': '2017-07-01T07:00:00Z', 'amount'...",NaN,Julie Keaton,892591.0,to buy more clothes to meet the needs and tast...,NaN,NaN,NaN,NaN
4,4,Farming,0.0,False,"[{'first_name': 'Alice', 'last_name': '', 'gen...",NaN,['en'],Alice (the woman pictured above in her small s...,NaN,NaN,NaN,0,NaN,1291518,2515975,1,0,0,0,400,Kenya,KE,town,1 38,point,Nandi Hills,Alice,156.0,[],2017-06-08T00:20:03Z,2017-05-09T00:20:03Z,Agriculture,fundraising,[{'name': '#Woman Owned Biz'}],40000.0,KES,2017-05-27T07:00:00Z,400,"[{'due_date': '2017-05-27T07:00:00Z', 'amount'...",shared,0.1,lender,Monthly,13,"[{'due_date': '2017-07-01T07:00:00Z', 'amount'...",['Rural Exclusion'],NaN,NaN,"to buy farming inputs (fertilizers, pesticides...",NaN,NaN,NaN,NaN


In [54]:
df=df[df['partner_id'].notnull()]

In [59]:
df['partner_id']=df['partner_id'].astype(int).astype(str)

In [60]:
partner_id=df['partner_id'].unique().tolist()

In [61]:
partner_id

['386',
 '156',
 '133',
 '218',
 '388',
 '164',
 '138',
 '202',
 '258',
 '262',
 '526',
 '520',
 '466',
 '477',
 '397',
 '512',
 '505',
 '500',
 '502',
 '436',
 '491',
 '469',
 '276',
 '322',
 '473',
 '405',
 '340',
 '350',
 '341',
 '412',
 '286',
 '447',
 '277',
 '324',
 '282',
 '213',
 '385',
 '259',
 '257',
 '212',
 '203',
 '336',
 '345',
 '272',
 '142',
 '24',
 '36',
 '92',
 '27',
 '25',
 '6',
 '32']

In [28]:
d = r.get('https://api.kivaws.org/v1/partners/322.json?app_id=org.deltanalytics')


In [29]:
d.headers

{'Date': 'Tue, 30 May 2017 07:02:52 GMT', 'Server': 'Apache/2.4.7 (Ubuntu)', 'Access-Control-Allow-Origin': '*', 'Expires': 'Tue, 03 Jul 2001 06:00:00 GMT', 'Last-Modified': 'Tue, 30 May 2017 07:02:52 GMT', 'Cache-Control': 'private, no-store, no-cache, must-revalidate, max-age=0, post-check=0, pre-check=0, proxy-revalidate, no-transform', 'Pragma': 'no-cache', 'X-RateLimit-Overall-Limit': '500', 'X-RateLimit-Overall-Remaining': '500', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '1255', 'Content-Type': 'application/json; charset=UTF-8'}

In [30]:
d.json()

{'lenders': [{'country_code': 'AU',
   'image': {'id': 2543858, 'template_id': 1},
   'lender_id': 'melbournesolar4571',
   'name': 'Melbourne Solar Solutions',
   'uid': 'melbournesolar4571',
   'whereabouts': 'Dandenong VIC'},
  {'image': {'id': 2543855, 'template_id': 1},
   'lender_id': 'shahab7717',
   'name': 'Shahab',
   'uid': 'shahab7717',
   'whereabouts': ''},
  {'country_code': 'GB',
   'image': {'id': 2543849, 'template_id': 1},
   'lender_id': 'essai7600',
   'name': 'Essai',
   'uid': 'essai7600',
   'whereabouts': ''},
  {'country_code': 'US',
   'image': {'id': 2543845, 'template_id': 1},
   'lender_id': 'drlinda5264',
   'name': 'Dr. Linda Makuta, DDS',
   'uid': 'drlinda5264',
   'whereabouts': 'Encino CA'},
  {'image': {'id': 726677, 'template_id': 1},
   'lender_id': 'andy1386',
   'name': 'andy',
   'uid': 'andy1386',
   'whereabouts': ''},
  {'image': {'id': 726677, 'template_id': 1},
   'lender_id': 'frances9054',
   'name': 'Frances',
   'uid': 'frances9054',
 

In [74]:
def extract_loan_partners(partner_ids):
    loan_partner_details=pd.DataFrame()
    for n in partner_ids:
        print (n)
        d = r.get('https://api.kivaws.org/v1/partners/'+n+'.json?app_id=org.deltanalytics')
        data = json.loads(d.text)
        partners=json_normalize(data['partners'])
        loan_partner_details=loan_partner_details.append(partners, ignore_index=True)
        print(len(loan_partner_details.index))
    
    return loan_partner_details  

In [75]:
loan_partner_details=extract_loan_partners(partner_id)

386
1
156
2
133
3
218
4
388
5
164
6
138
7
202
8
258
9
262
10
526
11
520
12
466
13
477
14
397
15
512
16
505
17
500
18
502
19
436
20
491
21
469
22
276
23
322
24
473
25
405
26
340
27
350
28
341
29
412
30
286
31
447
32
277
33
324
34
282
35
213
36
385
37
259
38
257
39
212
40
203
41
336
42
345
43
272
44
142
45
24
46
36
47
92
48
27
49
25
50
6
51
32
52


In [77]:
loan_partner_details.to_csv('~/intro_to_machine_learning/data/loans_partner_details.csv')

## Pulling each loan's lender details

Now that we have selected a subset of loans to focus in on, we want to pull all of the lenders of these loans. We do so here by first creating a list of the loans we are interested in. 

In [12]:
loan_ids = df['id'].astype(int).astype(str)
loan_ids = loan_ids.unique().tolist()
loan_ids[0:5]

['1291548', '1291532', '1291530', '1291525', '1291518']

Here we define a function that will extract details of every lender who has contributed to a single loan, and then loop it over the list of loans that we created in the previous step:

In [25]:
def extract_loan_lenders(loan_ids):
    loan_lenders_details=pd.DataFrame()
    for n in loan_ids:
        print (n)
        d = r.get('https://api.kivaws.org/v1/loans/'+n+'/lenders.json?app_id=org.deltanalytics')
        data = json.loads(d.text)
        if len(data['lenders']) != 0:
            lenders=json_normalize(data['lenders'])
        else: lenders = pd.DataFrame()
        lenders['loan_id'] = n
        loan_lenders_details = loan_lenders_details.append(lenders, ignore_index=True)
        print(len(loan_lenders_details.index))

    return loan_lenders_details  

In [ ]:
loan_lenders_details = extract_loan_lenders(loan_ids)

1291548
11
1291532
30
1291530
33
1291525
35
1291518
37
1291513
41
1291516
47
1291490
63
1291494
65
1291486
67
1291483
75
1291478
87
1291468
97
1291463
105
1291423
120
1291444
137
1291449
143
1291450
145
1291421
145
1291399
146
1291416
150
1291404
154
1291407
158
1291389
158
1291241
188
1291387
196
1291369
202
1291215
212
1291218
231
1291355
242
1291347
252
1291328
255
1291304
263
1291301
284
1291284
298
1291274
314
1291096
317
1291206
335
1291104
348
1291095
356
1291166
368
1291144
397
1291145
402
1291129
407
1291120
417
1291121
424
1291114
426
1291075
428
1291078
431
1291092
443
1291044
445
1291071
448
1291025
458
1291062
461
1291068
463
1291048
466
1291052
468
1288706
518
1291023
530
1291022
535
1290690
537
1290689
543
1290693
547
1290519
553
1290518
555
1290523
557
1290685
568
1290514
574
1290534
576
1290682
579
1290683
587
1290655
593
1290680
616
1290494
619
1290661
623
1290665
627
1290670
628
1290672
636
1290674
643
1290676
657
1290662
660
1290664
660
1290671
661
1290650
686
12906

We then write the output to a data frame. Let's take a look at what we've got! 

In [27]:
loan_lenders_details.head(100)

,country_code,image.id,image.template_id,lender_id,name,uid,whereabouts,loan_id
0,US,977510,1,tom7592,tom,tom7592,hilliard OH,1291548
1,AU,623712,1,ashley74191869,Ashley & Vicky,ashley74191869,Avalon NSW,1291548
2,NaN,1158966,1,heather3547,Heather,heather3547,,1291548
3,SE,1102204,1,magnusang,Magnus Äng,magnusang,Malmoe,1291548
4,NO,659790,1,helge1340,Helge,helge1340,Hærland,1291548
5,NaN,726677,1,100ofhumanity1199,100% of Humanity,100ofhumanity1199,,1291548
6,NaN,1525019,1,tasia6880,Tasia,tasia6880,,1291548
7,DE,432469,1,dirk1968,Dirk,dirk1968,Bonn,1291548
8,US,1343772,1,beth3603,Beth Ann,beth3603,Mason City Iowa,1291548
9,US,726677,1,will8174,Will,will8174,Houston TX,1291548


In [ ]:
loan_lenders_details.to_csv('~/intro_to_machine_learning/data/loans_lenders_details.csv')